In [1]:
from Tools.utils import *

In [8]:
# functions for different vector field transfers
# for this experiment we use FAUST original to have shapes in 1-1 correspondence
# so that computing ground truth vector field transfer is easier

# real representation for vector fields
def Real_rep_VF(self, k, kO, C = None):
    #kO = 30
    Ds = []
    for ik in range (kO):
        #ik = 8
        D = self.VF_fun_scal_op(self.ceig[:,ik], k)
        D_ = self.VF_fun_scal_op(1j*self.ceig[:,ik], k)
        if C is not None:
            D = D @ C
            D_= D_@ C
        Ds += [D, D_]
    Ds = np.stack(Ds, axis=0)
    return Ds

# transfer from Azencot et al.
def Omri_transfer(X, Y, vfx, C, kc):

    #print(vfx.shape)
    #print(X.eig_trans.shape)
    vfx_Op = X.VF_fun_scal_op(vfx, kr)
    Bvf = (Cr @ vfx_Op).flatten()
    #reshape([1, vfbasis.shape[1]*vfbasis.shape[2]])
    #Bvf = Bvf.T[:,0]
    #
    vfbasis = Real_rep_VF(Y, kr, kc, C=Cr)
    Avf = vfbasis.reshape([vfbasis.shape[0], vfbasis.shape[1]*vfbasis.shape[2]])
    #print(Avf.shape, Bvf.shape)
    vfy = np.linalg.pinv(Avf.T)@Bvf

    idv = np.arange(kc)
    vfy_a = vfy[2*idv]; vfy_b = vfy[2*idv+1];
    vfy = vfy_a + 1j*vfy_b
    vfy = Y.ceig[:,:kc] @ vfy
    
    return vfy

# Our transfer
def Q_transfer(X,Y, vfx, Q):
    k_Q = Q.shape[0]
    vfy = Y.ceig[:,:k_Q] @ Q @ np.linalg.pinv(X.ceig[:,:k_Q])@vfx
    return vfy

# transfer from Wang et al. (simply transfers gradients)
def grad_transfer(X,Y, sp, spi, C):
    k = sp.shape[0]
    C_ = C[:k,:k]
    #print(sp.shape, spi.shape, C_.shape)
    vfy = Y.grad_vert(Y.eig[:,:k] @ C_ @ sp) + 1j * Y.grad_vert(Y.eig[:,:k] @ C_ @ spi)
    return vfy

# get the real deformation X->Y (R2-tensor field for the pushforward in general)
# this function works only for meshes in 1-1 correspondence
def deformation_R2(self, Y):
    #
    Vjs = []
    Wjs = []
    rotate = np.zeros(self.v.shape[0], dtype=int)  #will keep in degree of vertex
    he = np.copy(self.he_start)
    i = 0
    while np.any(rotate==0):
        #-- get [vj] and store it in X; same for fj - fi in f
        lij = np.linalg.norm(self.v[self.e[he][:,1]] - self.v[self.e[he][:,0]], axis=1)
        aij = self.he_angles_norm[he]
        vj = lij[:, None] * np.cos(np.stack([aij, np.pi/2 - aij], axis=-1))

        #
        lij_ = np.linalg.norm(Y.v[self.e[he][:,1]] - Y.v[self.e[he][:,0]], axis=1)
        aij_ = Y.he_angles_norm[he]
        wj = lij_[:, None] * np.cos(np.stack([aij_, np.pi/2 - aij_], axis=-1))

        vj[rotate>0]=0 # do not add values if cycle done
        Vjs+=[vj]
        #
        wj[rotate>0]=0 # do not add values if cycle done
        Wjs+=[wj]

        #-- circulate CW
        he = self.op[he]
        he = self.nex[he]
        i+=1
        #-- update rot mask once cycle is done
        rot_mask = (self.he_start == he) * (rotate == 0)
        rotate[rot_mask] = i

    #-- build and invert local systems
    Vjs = np.stack(Vjs, axis=1)
    Vjs_inv = np.linalg.pinv(Vjs)

    Wjs = np.stack(Wjs, axis=1)
    
    return Vjs_inv @ Wjs

def cpl_to_real(vf):
    return np.stack([vf.real,vf.imag], axis=-1)
def real_to_cpl(vf):
    return vf[:,0] + 1j*vf[:,1]
def defoR2_vf(defo, vf):
    defovf = np.transpose(defo,(0,2,1)) @ cpl_to_real(vf)[:,:,None]
    return real_to_cpl(defovf)[:,0]

# load symmetric map on FAUST original dataset
T12_sym = np.loadtxt('data/FAUST_o/map_sym.map', dtype=int)

In [11]:
#50,58 -> 59 (SAME CHARACTER) and 52 -> 80,89 (OTHER CHARACTER)
pairs1 = np.stack([np.arange(50,59), 59* np.ones(9)]).T.astype(int)
pairs2 = np.stack([52* np.ones(10), np.arange(80,90)]).T.astype(int)
pairs = np.concatenate([pairs1, pairs2])

def one_norm(self, f):
    norm = np.linalg.norm((np.abs(np.sqrt(self.m) @ f)))
    area = np.sum(igl.doublearea(self.v, self.f)/2)
    #print(area)
    return norm/area

datafolder = 'data/FAUST_o'
vts_folder = 'cor'
filename = 'tr_reg_{:03d}.off'

errs_G = []
errs_O = []
errs_Q = []
for i1, i2 in pairs:
    print('**',i1,i2,'**')
    filename1 = join(datafolder, filename.format(i1))
    filename2 = join(datafolder, filename.format(i2))

    ## load (and save) spectral data
    k = 30
    ## compute real and complex eigenvectors
    X = mesh(filename1, spectral=k)
    Y = mesh(filename2, spectral=k)
    ##
    X.complex_spectral(k=k)
    Y.complex_spectral(k=k)
    #
    X.gradv = X.grad_vert_op()
    Y.gradv = Y.grad_vert_op()
    
    # load a specific vector field with its two components (random part)
    kr = k
    kc = kr
    C12_gt = np.linalg.pinv(Y.eig[:,:kr]) @ X.eig[:, :kr]
    C12_sym = pMap2fMap(Y.eig[:,:kr], X.eig[:,:kr], T12_sym)
    
    
    #(noise for fmap)
    # ******************* random noise ******************** 
    #noise = 2 * np.random.rand(kr*kr).reshape(kr,kr) - 1
    
    # ****************** symmetric noise ****************** 
    noise = C12_sym
    
    # random vector field
    #sp = (np.random.rand(kr)/(np.arange(kr)+1))#**2) ##less radial to look more rotational
    #sp[:3] = 0  ## cut low frequency
    #fx = X.eig[:,:kr] @ sp
    #
    #spi = (np.random.rand(kr)/(np.arange(kr)+1))
    #spi[:3] = 0
    #fxi = X.eig[:,:kr] @ spi
    
    # asymmetric vf
    sp = np.linalg.pinv(X.eig[:,:kr]) @ X.v[:,0]
    spi = 0 * sp
    fx = X.eig[:,:kr] @ sp
    fxi = X.eig[:,:kr] @ spi
    
    # compute transfer with different method (here for random noise)
    #Cr = C12_gt[:kr, :kr]
    #Cr = C12_gt[:kr, :kr] + 1/5*noise
    
    a = 0.5  # (here for symmetric noise)
    Cr = (1-a) * C12_gt + a * noise
    Qc = CMap2QMap_procrustes(X,Y,Cr,kc)
    
    # assemble vector field with Hodge decomposition
    vfx = X.grad_vert(fx) + 1j * X.grad_vert(fxi)
    ##

    vfy_g = grad_transfer(X, Y, sp, spi, Cr)
    vfy_o = Omri_transfer(X, Y, vfx, Cr, kc)
    vfy_q = Q_transfer(   X, Y, vfx, Qc)
    
    #computing GT with defo
    defo = deformation_R2(X,Y)
    vfy_gt = defoR2_vf(defo, vfx)
    
    ##
    fG = vfy_g - vfy_gt
    fO = vfy_o - vfy_gt
    fQ = vfy_q - vfy_gt
    
    Nvf = one_norm(Y, vfx)
    
    errG = one_norm(Y, fG)/Nvf
    errO = one_norm(Y, fO)/Nvf
    errQ = one_norm(Y, fQ)/Nvf
    errs_G += [errG]; errs_O += [errO]; errs_Q += [errQ]
    
    print('norm', Nvf)
    #print('mult', one_norm(Y, mult))
    print("***")

    print('grad', errG)
    print('Omri', errO)
    print('Qtra', errQ)
    print("***") 

** 50 59 **
norm 0.46305344132525356
***
grad 1.1130440081732014
Omri 11.97919142955905
Qtra 0.6659146179606849
***
** 51 59 **
norm 0.48019036780754853
***
grad 1.0536597499855538
Omri 5.3564259777708765
Qtra 0.9494306576271175
***
** 52 59 **
norm 0.44323986395074794
***
grad 1.1012348110014172
Omri 7.038134235561935
Qtra 0.5271180067259283
***
** 53 59 **
norm 0.4407539055731817
***
grad 1.049384086278919
Omri 4.3054935317836325
Qtra 0.5743576535438795
***
** 54 59 **
norm 0.44335223368048926
***
grad 1.1191920738161696
Omri 6.223959046863097
Qtra 0.6351084955868053
***
** 55 59 **
norm 0.45693841955855946
***
grad 1.0724497085207436
Omri 16.603195864397193
Qtra 0.6324530864987351
***
** 56 59 **
norm 0.44230440807411736
***
grad 1.0564658746311353
Omri 4.898704560735392
Qtra 0.4981492901357069
***
** 57 59 **
norm 0.5263617944232273
***
grad 0.9293886659252182
Omri 9.265150893170546
Qtra 0.475916478922694
***
** 58 59 **
norm 0.4567598978489657
***
grad 0.777887260409518
Omri 7.615

In [12]:
print('Wang', np.round(np.mean(errs_G),2))
print('Azen', np.round(np.mean(errs_O),2))
print('Ours', np.round(np.mean(errs_Q),2))

Wang 1.13
Azen 6.84
Ours 0.62
